In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Carregue seus dados em um DataFrame
data = pd.read_csv('Dataset/Dados da Torre de Santarém com NPP - Dados da Torre de Santarém com NPP.csv.csv')

# Divida os dados em recursos (X) e rótulo (y)
X = data[['Tair_f', 'VPD_f', 'rH', 'Year', 'H_f', 'FP_Temp_uStar', 'Tsoil_f', 'LE_f', 'Rg_f']]
y = data['NPP']

# Padronize os recursos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Treine o modelo GBM
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
gbm.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = gbm.predict(X_test)

# Calcule as métricas de avaliação
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE (Mean Squared Error): {mse}')
print(f'MAE (Mean Absolute Error): {mae}')
print(f'R-squared (R2 Score): {r2}')

# Calcule a importância das características
feature_importance = gbm.feature_importances_

# Crie um gráfico de barras para mostrar a importância das características
plt.bar(X.columns, feature_importance)
plt.xlabel('Variáveis')
plt.ylabel('Importância')
plt.title('Importância das Variáveis em Relação ao NPP')
plt.xticks(rotation=45)
plt.show()



C:\Users\leona\AppData\Local\Temp\ipykernel_7644\1934703044.py:10: DtypeWarning: Columns (26,94) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:/Users/leona/OneDrive/Documentos/Projeto MAI-DAI/Dados da Torre de Santarém com NPP - Dados da Torre de Santarém com NPP.csv.csv')


ValueError: could not convert string to float: '1.990.386'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


# Carregue seus dados em um DataFrame
data = pd.read_csv('Dataset/Dados da Torre de Santarém com NPP - Dados da Torre de Santarém com NPP.csv.csv')


# Divida os dados em recursos (X) e rótulo (y)
X = data[['Tair_f', 'VPD_f', 'rH', 'Year', 'H_f','FP_Temp_uStar','Tsoil_f', 'LE_f','Rg_f']]
y = data['NPP']


# Padronize os recursos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Treine o modelo GBM
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
gbm.fit(X_scaled, y)


# Calcule a importância das características
feature_importance = gbm.feature_importances_


mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print(f'MSE (Mean Squared Error): {mse}')
print(f'MAE (Mean Absolute Error): {mae}')
print(f'R-squared (R2 Score): {r2}')




# Crie um gráfico de barras para mostrar a importância das características
plt.bar(X.columns, feature_importance)
plt.xlabel('Variáveis')
plt.ylabel('Importância')
plt.title('Importância das Variáveis em Relação ao NPP')
plt.xticks(rotation=45)
plt.show()


C:\Users\leona\AppData\Local\Temp\ipykernel_11152\3454682065.py:9: DtypeWarning: Columns (26,94) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Dataset/Dados da Torre de Santarém com NPP - Dados da Torre de Santarém com NPP.csv.csv')


ValueError: could not convert string to float: '1.990.386'